In [20]:
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
dataset=pd.read_csv('insurance_pre.csv')
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [31]:
dataset= pd.get_dummies(dataset,drop_first=True)
dataset=dataset.replace({True:1,False:0})
dataset

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,16884.92400,1,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0
2,28,33.000,3,4449.46200,0,1,1,0
3,33,22.705,0,21984.47061,0,1,1,0
4,32,28.880,0,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0
1334,18,31.920,0,2205.98080,1,0,1,0
1335,18,36.850,0,1629.83350,1,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0


In [32]:
independent= dataset[['age', 'bmi', 'children','sex_male','smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [33]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid= {'criterion':['squared_error','absolute_error'],'n_estimators':[50,100]}
grid= GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,),

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'n_estimators': [50, 100]},
             verbose=3)

In [62]:
re= grid.cv_results_
print("the finalized and best score was:",format(grid.best_params_))

the finalized and best score was: {'criterion': 'squared_error', 'n_estimators': 100}


In [63]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.082510,0.005700,0.002559,0.000127,squared_error,50,"{'criterion': 'squared_error', 'n_estimators':...",0.858652,0.759771,0.812901,0.796580,0.773457,0.800272,0.034469,2
1,0.133698,0.006399,0.004309,0.000117,squared_error,100,"{'criterion': 'squared_error', 'n_estimators':...",0.863646,0.768880,0.811927,0.805542,0.767702,0.803539,0.035136,1
2,0.338746,0.007709,0.003300,0.000856,absolute_error,50,"{'criterion': 'absolute_error', 'n_estimators'...",0.835832,0.767695,0.793720,0.791994,0.764363,0.790721,0.025576,4
3,0.626596,0.007214,0.004207,0.000253,absolute_error,100,"{'criterion': 'absolute_error', 'n_estimators'...",0.850072,0.783863,0.795152,0.800962,0.769584,0.799927,0.027266,3


In [38]:
age_input=int(input("Age:"))
bmi_input=int(input("BMI:"))
children=int(input("Children:"))
isMale=int(input("Male:"))
smoker=int(input("Smoker:"))

Age: 22
BMI: 27
Children: 3
Male: 0
Smoker: 0


In [64]:
Future_Prediction= grid.predict([[20,24,1,0,0]])
print("Future prediction was:",format(Future_Prediction))

Future prediction was: [3781.3757983]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
